# API Client
The following tutorial demonstrates how to access the PKDB API with python.

For more information and installation information see:  
https://github.com/matthiaskoenig/pkdb

In [1]:
URL_PKDB = "http://0.0.0.0:8000/"

In [2]:
# import libraries
import coreapi
import requests
import json
from pprint import pprint
from requests.auth import HTTPBasicAuth

In [3]:
#response = requests.get('http://0.0.0.0:8000/', auth=HTTPBasicAuth('janek', 'test'))

In [4]:
#r = requests.post("http://0.0.0.0:8000/api-auth/login/?next=/api/v1/", data={'username': 'janek', 'password': 'test'})
#print(r.status_code, r.reason)
#r.json()["token"]

## Create client & API endpoint

In [5]:
#auth = coreapi.auth.BasicAuthentication(
#username="janek",
#password="test", )
client = coreapi.Client()

In [6]:
# get the api scema
document = client.get(URL_PKDB)
print(document.title)
print(document.url)

PKDB API
http://0.0.0.0:8000/


In [7]:
# overview available methods
print(document)

<PKDB API "http://0.0.0.0:8000/">
    authors: {
        list([page], [first_name], [last_name], [search])
        create([first_name], [last_name])
        read(id, [first_name], [last_name], [search])
        update(id, [first_name], [last_name], [first_name], [last_name], [search])
        partial_update(id, [first_name], [last_name], [first_name], [last_name], [search])
        delete(id, [first_name], [last_name], [search])
    }
    intervention: {
        list([page], [comment], [description], [type], [search])
        create(type, [comment], [description], [study])
        read(id, [comment], [description], [type], [search])
        update(id, type, [comment], [description], [study], [comment], [description], [type], [search])
        partial_update(id, [comment], [description], [type], [study], [comment], [description], [type], [search])
        delete(id, [comment], [description], [type], [search])
    }
    studies: {
        list([page], [comment], [description], [title], [

## Example queries
Show all authors

In [8]:
data = client.action(document,["authors", "list"])
print(json.dumps(data, indent=2))

{
  "count": 1,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    }
  ]
}


Create author

In [9]:
data = client.action(document,["authors", "create"], params={'first_name':'Lucek', 'last_name':'Grzegorzewski'})
print(data)

OrderedDict([('id', 1), ('first_name', 'Lucek'), ('last_name', 'Grzegorzewski')])


In [13]:
# Filter for authors first name = Lucek
data = client.action(document, ["authors", "list"], params={"first_name": "Lucek"} )
print(json.dumps(data, indent=2))

{
  "count": 1,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    }
  ]
}


In [14]:
# Search for author
data = client.action(document, ["authors", "list"], params={"search": "Gr"} )
print(json.dumps(data, indent=2))

{
  "count": 3,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    },
    {
      "id": 16,
      "first_name": "Marika T",
      "last_name": "Granfors"
    },
    {
      "id": 42,
      "first_name": "L",
      "last_name": "Granit"
    }
  ]
}


In [12]:
# Create study
study_dict = {
    'title': "4-quinolones inhibit joiijojiojioijojioji of caffeine.", 
    'pmid': '19125908', 
    'authors':[  
      {'first_name': 'C-Y', 'last_name': 'Fun'}, 
      {'first_name': 'Y-L', 'last_name': 'Law'}, 
      {'first_name': 'W-M', 'last_name': 'Lim'},
      {'first_name': 'W', 'last_name': 'Fan'}, 
      {'first_name': 'C-L', 'last_name': 'Lim'}
        
    ]
}
client.action(document,["studies", "create"], params=study_dict)

OrderedDict([('comment', None),
             ('description', None),
             ('title',
              '4-quinolones inhibit joiijojiojioijojioji of caffeine.'),
             ('pmid', '19125908'),
             ('authors',
              [OrderedDict([('id', 2),
                            ('first_name', 'C-Y'),
                            ('last_name', 'Fun')]),
               OrderedDict([('id', 3),
                            ('first_name', 'Y-L'),
                            ('last_name', 'Law')]),
               OrderedDict([('id', 4),
                            ('first_name', 'W-M'),
                            ('last_name', 'Lim')]),
               OrderedDict([('id', 5),
                            ('first_name', 'W'),
                            ('last_name', 'Fan')]),
               OrderedDict([('id', 6),
                            ('first_name', 'C-L'),
                            ('last_name', 'Lim')])]),
             ('abstract', None),
             ('file', None)])